In [1]:
# import libraries
import keras
from utils.preprocess import *

In [2]:
# config
no_cap = False  # if True, capital letters will be ignored
train_prop = 0.7  # proportion of training data
batch_size = 32
shuffle_buffer_size = 10000
epochs = 10

target_model = "unipen_no_cap_model" if no_cap else "unipen_model"

In [3]:
# load dataset
dataset = load_unipen_dataset(no_cap).shuffle(shuffle_buffer_size)

# normalize dataset
def normalize(data, label):
    return tf.cast(data, tf.float32) / 255.0, label

dataset = dataset.map(normalize)

# split & filter dataset
train_size = int(train_prop * dataset.cardinality().numpy())
train_dataset = dataset.take(train_size).batch(batch_size)
test_dataset = dataset.skip(train_size).batch(batch_size)

In [4]:
# build model
model = keras.Sequential([
    keras.layers.Conv2D(32, (4, 4), activation='relu', input_shape=(64, 64, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    # keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128)
])
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [5]:
model.fit(train_dataset, epochs=epochs)

Epoch 1/10
1365/1365 [==============================] - 19s 14ms/step - loss: 1.3454 - accuracy: 0.6402
Epoch 2/10
1365/1365 [==============================] - 20s 15ms/step - loss: 0.6696 - accuracy: 0.7964
Epoch 3/10
1365/1365 [==============================] - 20s 15ms/step - loss: 0.4376 - accuracy: 0.8636
Epoch 4/10
1365/1365 [==============================] - 20s 15ms/step - loss: 0.2948 - accuracy: 0.9057
Epoch 5/10
1365/1365 [==============================] - 20s 15ms/step - loss: 0.2051 - accuracy: 0.9356
Epoch 6/10
1365/1365 [==============================] - 20s 15ms/step - loss: 0.1564 - accuracy: 0.9507
Epoch 7/10
1365/1365 [==============================] - 21s 15ms/step - loss: 0.1202 - accuracy: 0.9636
Epoch 8/10
1365/1365 [==============================] - 21s 16ms/step - loss: 0.1021 - accuracy: 0.9699
Epoch 9/10
1365/1365 [==============================] - 21s 15ms/step - loss: 0.0841 - accuracy: 0.9750
Epoch 10/10
1365/1365 [==============================] - 21s 15m

In [6]:
# evaluate model
test_loss, test_acc = model.evaluate(test_dataset)

print()
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

585/585 [==============================] - 3s 5ms/step - loss: 0.8271 - accuracy: 0.8408

Test loss: 0.8270606398582458
Test accuracy: 0.8407694101333618


In [7]:
# save model
model.save(f"data/{target_model}.h5")

/Users/dogeon/Documents/code/py/unipen/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
